In [1]:
from wikidataintegrator import wdi_core, wdi_login
from wikidataintegrator.ref_handlers import update_retrieved_if_new_multiple_refs
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from time import sleep
from random import random

In [2]:
kantonZH = pd.read_csv('kantonZH.csv', parse_dates=['date'])    
kantonZH.head()
#[(city['Name']=='Adlikon')]
cityZH = pd.read_csv('cityZH.csv', parse_dates=['date'])    
cityZH.head()
df = cityZH

print("Logging in...")
#if "WDUSER" in os.environ and "WDPASS" in os.environ:
#    WDUSER = os.environ['WDUSER']
#    WDPASS = os.environ['WDPASS']
#else:
#    raise ValueError("WDUSER and WDPASS must be specified in local.py or as environment variables")
user = "Tlzh"
password = "wikiwiki"
login = wdi_login.WDLogin(user, password)

Logging in...
https://www.wikidata.org/w/api.php
Successfully logged in as Tlzh


In [3]:
# Quelle setzen
# Quartier 
#  Q4115189|P1082|805|P585|+2017-12-31T00:00:00Z/9|P459|Q12483|S854|"https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_seit1970_od3240"
from datetime import datetime
import copy
def create_reference():
    timeStringNow = datetime.now().strftime("+%Y-%m-%dT00:00:00Z")
    # Timestamp P585
    refRetrieved = wdi_core.WDTime(timeStringNow, prop_nr="P813", is_reference=True)
    # URL Source P854
    refURL = wdi_core.WDUrl(value="https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_seit1970_od3240", 
                            prop_nr="P854", is_reference=True)

    return [refRetrieved, refURL]

def create_qualifier(insertDate):
    # Point in time P585
    insertDate = insertDate.strftime("+%Y-%m-%dT00:00:00Z")
    refRetrieved = wdi_core.WDTime(insertDate, prop_nr="P585", is_qualifier=True)
    # Statistics P459
    bestimmungsmethode = wdi_core.WDItemID(value='Q12483', prop_nr='P459', is_qualifier=True)

    return [refRetrieved, bestimmungsmethode]


In [4]:
df = df[df['bool']==False]
print("Number of inserts: ", str(df['bool'].count()))
for i, row in df.iterrows():
    print("Insert data value: {0} - {1} - {2} : {3}".format(row['wikidata_id'],row['name'], row['date'], row['population']))
    reference = create_reference()
    insertDate = datetime(2017,12,31).strftime("+%Y-%m-%dT00:00:00Z")
    qualfier = create_qualifier(row['date'])
    statement = [wdi_core.WDQuantity(value=row['population'], prop_nr="P1082", references=[copy.deepcopy(reference)], 
                                     rank='normal', qualifiers=copy.deepcopy(qualfier), check_qualifier_equality=False)]
    wd_item = wdi_core.WDItemEngine(data=statement,wd_item_id=row['wikidata_id'], new_item=False, fast_run=False, append_value=['P1082'])
    wd_item.write(login)

Number of inserts:  34
Insert data value: Q693374 - Hirzenbach - 2018-12-31 00:00:00 : 12801
Insert data value: Q692728 - Schwamendingen-Mitte - 2018-12-31 00:00:00 : 11100
Insert data value: Q652455 - Saatlen - 2018-12-31 00:00:00 : 8582
Insert data value: Q276792 - Seebach - 2018-12-31 00:00:00 : 25568
Insert data value: Q167179 - Oerlikon - 2018-12-31 00:00:00 : 23214
Insert data value: Q382903 - Affoltern - 2018-12-31 00:00:00 : 26562
Insert data value: Q678030 - Wipkingen - 2018-12-31 00:00:00 : 16321
Insert data value: Q445711 - Altstetten - 2018-12-31 00:00:00 : 33461
Insert data value: Q80797 - Albisrieden - 2018-12-31 00:00:00 : 22304
Insert data value: Q693321 - Weinegg - 2018-12-31 00:00:00 : 5220
Insert data value: Q693397 - Mühlebach - 2018-12-31 00:00:00 : 6315
Insert data value: Q692773 - Seefeld - 2018-12-31 00:00:00 : 5253
Insert data value: Q392079 - Witikon - 2018-12-31 00:00:00 : 10953
Insert data value: Q476940 - Hirslanden - 2018-12-31 00:00:00 : 7488
Insert data 